In [ ]:
#v1

# resnet18_training.py

import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, datasets, transforms
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
from tqdm import tqdm
import matplotlib.pyplot as plt
from sklearn.utils.class_weight import compute_class_weight
import numpy as np
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# Paths
DATASET_DIR = "../data/processed"
train_dir = os.path.join(DATASET_DIR, "train")
val_dir = os.path.join(DATASET_DIR, "val")
test_dir = os.path.join(DATASET_DIR, "test")
# Transforms for ResNet18
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])
# Load datasets
datasets_dict = {
    'train': datasets.ImageFolder(train_dir, transform=transform),
    'val': datasets.ImageFolder(val_dir, transform=transform),
    'test': datasets.ImageFolder(test_dir, transform=transform)
}
# DataLoaders
dataloaders = {
    x: DataLoader(datasets_dict[x], batch_size=32, shuffle=(x == 'train'), num_workers=2)
    for x in ['train', 'val', 'test']
}

class_names = datasets_dict['train'].classes
num_classes = len(class_names)
# === NEW: Compute class weights ===
train_labels = [label for _, label in datasets_dict['train']]
class_weights = compute_class_weight(class_weight='balanced', classes=np.arange(num_classes), y=train_labels)
class_weights_tensor = torch.tensor(class_weights, dtype=torch.float).to(device)
# Load pretrained ResNet18 and modify final layer
model = models.resnet18(pretrained=True)
for param in model.parameters():
    param.requires_grad = False  # Freeze all layers
model.fc = nn.Linear(model.fc.in_features, num_classes)
model = model.to(device)
# Use weighted loss
criterion = nn.CrossEntropyLoss(weight=class_weights_tensor)
optimizer = optim.Adam(model.fc.parameters(), lr=0.001)
# TensorBoard writer
writer = SummaryWriter(log_dir="runs/resnet18_experiment")
# Training loop
def train_model(model, dataloaders, criterion, optimizer, num_epochs=10):
    best_val_loss = float('inf')
    train_loss_hist, val_loss_hist = [], []

    for epoch in range(num_epochs):
        print(f"\nEpoch {epoch+1}/{num_epochs}")
        print("-" * 20)

        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()
            else:
                model.eval()

            running_loss = 0.0
            running_corrects = 0

            with tqdm(dataloaders[phase], unit="batch") as tepoch:
                for inputs, labels in tepoch:
                    inputs, labels = inputs.to(device), labels.to(device)

                    optimizer.zero_grad()

                    with torch.set_grad_enabled(phase == 'train'):
                        outputs = model(inputs)
                        loss = criterion(outputs, labels)
                        _, preds = torch.max(outputs, 1)

                        if phase == 'train':
                            loss.backward()
                            optimizer.step()

                    running_loss += loss.item() * inputs.size(0)
                    running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(datasets_dict[phase])
            epoch_acc = running_corrects.double() / len(datasets_dict[phase])

            print(f"{phase.capitalize()} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}")

            writer.add_scalar(f"Loss/{phase}", epoch_loss, epoch)
            writer.add_scalar(f"Accuracy/{phase}", epoch_acc, epoch)

            if phase == 'train':
                train_loss_hist.append(epoch_loss)
            else:
                val_loss_hist.append(epoch_loss)
                if epoch_loss < best_val_loss:
                    best_val_loss = epoch_loss
                    torch.save(model.state_dict(), "../model/best_resnet18_model.pth")

    print("\nTraining complete. Best validation loss: {:.4f}".format(best_val_loss))
    return train_loss_hist, val_loss_hist
# Train
train_loss_hist, val_loss_hist = train_model(model, dataloaders, criterion, optimizer, num_epochs=10)
# Evaluation
def evaluate_model(model, dataloader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            correct += torch.sum(preds == labels).item()
            total += labels.size(0)
    print(f"\nTest Accuracy: {100 * correct / total:.2f}%")
evaluate_model(model, dataloaders["test"])
# Plot loss curves
plt.figure(figsize=(10, 5))
plt.plot(train_loss_hist, label="Train Loss")
plt.plot(val_loss_hist, label="Val Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("Training and Validation Loss")
plt.legend()
plt.savefig("loss_curve.png")
plt.show()

writer.close()

In [ ]:
# v2
# resnet18_training.py

import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, datasets, transforms
from torchvision.models import ResNet18_Weights
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
from torch.optim.lr_scheduler import ReduceLROnPlateau
from tqdm import tqdm
import matplotlib.pyplot as plt
from sklearn.utils.class_weight import compute_class_weight
import numpy as np
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# Paths
DATASET_DIR = "../data/processed"
train_dir = os.path.join(DATASET_DIR, "train")
val_dir = os.path.join(DATASET_DIR, "val")
test_dir = os.path.join(DATASET_DIR, "test")
# Transforms for ResNet18
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])
# Load datasets
datasets_dict = {
    'train': datasets.ImageFolder(train_dir, transform=transform),
    'val': datasets.ImageFolder(val_dir, transform=transform),
    'test': datasets.ImageFolder(test_dir, transform=transform)
}
# DataLoaders
dataloaders = {
    x: DataLoader(datasets_dict[x], batch_size=32, shuffle=(x == 'train'), num_workers=4, prefetch_factor=4)
    for x in ['train', 'val', 'test']
}

class_names = datasets_dict['train'].classes
num_classes = len(class_names)
# === NEW: Compute class weights ===
train_labels = [label for _, label in datasets_dict['train']]
class_weights = compute_class_weight(class_weight='balanced', classes=np.arange(num_classes), y=train_labels)
class_weights_tensor = torch.tensor(class_weights, dtype=torch.float).to(device)
# Load pretrained ResNet18 and modify final layer
model = models.resnet18(weights=ResNet18_Weights.DEFAULT)
for name, param in model.named_parameters():
    if "layer4" in name or "fc" in name:
        param.requires_grad = True
    else:
        param.requires_grad = False
# todo take this out later
# for param in model.parameters():
#     param.requires_grad = False  # Freeze all layers
model.fc = nn.Linear(model.fc.in_features, num_classes)
model = model.to(device)
# Use weighted loss
criterion = nn.CrossEntropyLoss(weight=class_weights_tensor)
# Get trainable parameters only
trainable_params = filter(lambda p: p.requires_grad, model.parameters())
optimizer = optim.Adam(trainable_params, lr=0.001)
scheduler = ReduceLROnPlateau(optimizer, mode='min', patience=2, factor=0.5)
# TensorBoard writer
writer = SummaryWriter(log_dir="runs/resnet18_experiment_2")
# Training loop
def train_model(model, dataloaders, criterion, optimizer, num_epochs=10):
    best_val_loss = float('inf')
    train_loss_hist, val_loss_hist = [], []
    train_acc_hist, val_acc_hist = [], []

    for epoch in range(num_epochs):
        print(f"\nEpoch {epoch+1}/{num_epochs}")
        print("-" * 20)

        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()
            else:
                model.eval()

            running_loss = 0.0
            running_corrects = 0

            with tqdm(dataloaders[phase], unit="batch") as tepoch:
                for i, (inputs, labels) in enumerate(tepoch):
                    # Optional: show batch progress every 10 batches
                    # if i % 10 == 0:  # 🔍 Debug: monitor training progress
                    #     print(f"[{phase.upper()}] Epoch {epoch+1} - Batch {i+1}/{len(dataloaders[phase])}")  # REMOVE LATER

                    inputs, labels = inputs.to(device), labels.to(device)

                    optimizer.zero_grad()

                    with torch.set_grad_enabled(phase == 'train'):
                        outputs = model(inputs)
                        loss = criterion(outputs, labels)
                        _, preds = torch.max(outputs, 1)

                        if phase == 'train':
                            loss.backward()
                            optimizer.step()

                    running_loss += loss.item() * inputs.size(0)
                    running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(datasets_dict[phase])
            epoch_acc = running_corrects.double() / len(datasets_dict[phase])

            print(f"{phase.capitalize()} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}")

            writer.add_scalar(f"Loss/{phase}", epoch_loss, epoch)
            writer.add_scalar(f"Accuracy/{phase}", epoch_acc, epoch)

            if phase == 'train':
                train_loss_hist.append(epoch_loss)
                train_acc_hist.append(epoch_acc.item())
            else:
                val_loss_hist.append(epoch_loss)
                val_acc_hist.append(epoch_acc.item())
                scheduler.step(epoch_loss)

                if epoch_loss < best_val_loss:
                    best_val_loss = epoch_loss
                    torch.save(model.state_dict(), "../model/dermai_model.pth")

    print("\nTraining complete. Best validation loss: {:.4f}".format(best_val_loss))
    return train_loss_hist, val_loss_hist, train_acc_hist, val_acc_hist
# Train
train_loss_hist, val_loss_hist, train_acc_hist, val_acc_hist = train_model(model, dataloaders, criterion, optimizer, num_epochs=10)
# Save complete model for inference
complete_model_path = "../model/dermai_model.pt"
torch.save(model, complete_model_path)
print(f"\nFinal model saved to {complete_model_path}")
# Evaluation
def evaluate_model(model, dataloader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            correct += torch.sum(preds == labels).item()
            total += labels.size(0)
    print(f"\nTest Accuracy: {100 * correct / total:.2f}%")
evaluate_model(model, dataloaders["test"])
# Plot loss curves
plt.figure(figsize=(10, 5))
plt.plot(range(1, len(train_loss_hist) + 1), train_loss_hist,label="Training Loss")
plt.plot(range(1, len(val_loss_hist) + 1), val_loss_hist,label="Validation Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("Training and Validation Loss")
plt.legend()
plt.savefig("loss_curve.png")
plt.show()
# Plot loss curves
plt.figure(figsize=(10, 5))
plt.plot(range(1, len(train_acc_hist) + 1), train_acc_hist,label="Training Accuracy")
plt.plot(range(1, len(val_acc_hist) + 1), val_acc_hist,label="Validation Accuracy")
plt.xlabel("Epoch")
plt.ylabel("accuracy")
plt.title("Training and Validation Accuracy")
plt.legend()
plt.savefig("accuracy_curve.png")
plt.show()

writer.close()

In [ ]:
#v3

# resnet18_training.py

import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, datasets, transforms
from torchvision.models import ResNet18_Weights
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
from torch.optim.lr_scheduler import ReduceLROnPlateau
from tqdm import tqdm
import matplotlib.pyplot as plt
from sklearn.utils.class_weight import compute_class_weight
import numpy as np
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# Paths
DATASET_DIR = "../data/processed"
train_dir = os.path.join(DATASET_DIR, "train")
val_dir = os.path.join(DATASET_DIR, "val")
test_dir = os.path.join(DATASET_DIR, "test")
# === Transforms ===
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])
# Load datasets
datasets_dict = {
    'train': datasets.ImageFolder(train_dir, transform=transform),
    'val': datasets.ImageFolder(val_dir, transform=transform),
    'test': datasets.ImageFolder(test_dir, transform=transform)
}
# DataLoaders
dataloaders = {
    x: DataLoader(datasets_dict[x], batch_size=32, shuffle=(x == 'train'), num_workers=4, prefetch_factor=4)
    for x in ['train', 'val', 'test']
}

class_names = datasets_dict['train'].classes
num_classes = len(class_names)
# === NEW: Compute class weights ===
train_labels = [label for _, label in datasets_dict['train']]
class_weights = compute_class_weight(class_weight='balanced', classes=np.arange(num_classes), y=train_labels)
class_weights_tensor = torch.tensor(class_weights, dtype=torch.float).to(device)
# === Load and Modify Pretrained Model ===
model = models.resnet18(weights=ResNet18_Weights.DEFAULT)

# ✅ Unfreeze layer3 + layer4 + fc
for name, param in model.named_parameters():
    if "layer3" in name or "layer4" in name or "fc" in name:
        param.requires_grad = True
    else:
        param.requires_grad = False

model.fc = nn.Linear(model.fc.in_features, num_classes)
model = model.to(device)
# ✅ Weighted Loss + Label Smoothing
criterion = nn.CrossEntropyLoss(weight=class_weights_tensor, label_smoothing=0.1)

# ✅ Add weight decay (L2 regularization)
trainable_params = filter(lambda p: p.requires_grad, model.parameters())
optimizer = optim.Adam(trainable_params, lr=0.001, weight_decay=1e-4)
scheduler = ReduceLROnPlateau(optimizer, mode='min', patience=2, factor=0.5)
# TensorBoard writer
writer = SummaryWriter(log_dir="runs/resnet18_experiment_v3")
# === Training Function with Early Stopping ===
def train_model(model, dataloaders, criterion, optimizer, num_epochs=20, patience=3):
    best_val_loss = float('inf')
    train_loss_hist, val_loss_hist = [], []
    train_acc_hist, val_acc_hist = [], []
    patience_counter = 0

    for epoch in range(num_epochs):
        print(f"\nEpoch {epoch+1}/{num_epochs}")
        print("-" * 20)

        for phase in ['train', 'val']:
            model.train() if phase == 'train' else model.eval()
            running_loss, running_corrects = 0.0, 0

            with tqdm(dataloaders[phase], unit="batch") as tepoch:
                for inputs, labels in tepoch:
                    inputs, labels = inputs.to(device), labels.to(device)
                    optimizer.zero_grad()

                    with torch.set_grad_enabled(phase == 'train'):
                        outputs = model(inputs)
                        loss = criterion(outputs, labels)
                        _, preds = torch.max(outputs, 1)

                        if phase == 'train':
                            loss.backward()
                            optimizer.step()

                    running_loss += loss.item() * inputs.size(0)
                    running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(datasets_dict[phase])
            epoch_acc = running_corrects.double() / len(datasets_dict[phase])

            print(f"{phase.capitalize()} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}")
            writer.add_scalar(f"Loss/{phase}", epoch_loss, epoch)
            writer.add_scalar(f"Accuracy/{phase}", epoch_acc, epoch)

            if phase == 'train':
                train_loss_hist.append(epoch_loss)
                train_acc_hist.append(epoch_acc.item())
            else:
                val_loss_hist.append(epoch_loss)
                val_acc_hist.append(epoch_acc.item())
                scheduler.step(epoch_loss)

                if epoch_loss < best_val_loss:
                    best_val_loss = epoch_loss
                    torch.save(model.state_dict(), "../model/dermai_model_v2.pth")
                    patience_counter = 0  # Reset patience if improved
                    print("✅ Best model updated.")
                else:
                    patience_counter += 1
                    print(f"⚠️ Patience: {patience_counter}/{patience}")
                    if patience_counter >= patience:
                        print("⏹️ Early stopping triggered.")
                        return train_loss_hist, val_loss_hist, train_acc_hist, val_acc_hist

    print("\n✅ Training complete. Best validation loss: {:.4f}".format(best_val_loss))
    return train_loss_hist, val_loss_hist, train_acc_hist, val_acc_hist
# === Train ===
train_loss_hist, val_loss_hist, train_acc_hist, val_acc_hist = train_model(
    model, dataloaders, criterion, optimizer, num_epochs=20, patience=5
)
# Save full model
complete_model_path = "../model/dermai_model_v2.pt"
torch.save(model, complete_model_path)
print(f"\n📦 Final model saved to {complete_model_path}")
# === Evaluation ===
def evaluate_model(model, dataloader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            correct += torch.sum(preds == labels).item()
            total += labels.size(0)
    print(f"\n🧪 Test Accuracy: {100 * correct / total:.2f}%")
evaluate_model(model, dataloaders["test"])
# Plot loss curves
plt.figure(figsize=(10, 5))
plt.plot(range(1, len(train_loss_hist) + 1), train_loss_hist,label="Training Loss")
plt.plot(range(1, len(val_loss_hist) + 1), val_loss_hist,label="Validation Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("Training and Validation Loss")
plt.legend()
plt.savefig("loss_curve_v2.png")
plt.show()
# Plot loss curves
plt.figure(figsize=(10, 5))
plt.plot(range(1, len(train_acc_hist) + 1), train_acc_hist,label="Training Accuracy")
plt.plot(range(1, len(val_acc_hist) + 1), val_acc_hist,label="Validation Accuracy")
plt.xlabel("Epoch")
plt.ylabel("accuracy")
plt.title("Training and Validation Accuracy")
plt.legend()
plt.savefig("accuracy_curve_v2.png")
plt.show()

writer.close()